In [86]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from utils.dnn import *
from utils.data_encoders import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## QNN

In [88]:
np.random.seed(42)
backend = Aer.get_backend('qasm_simulator')

layer1 = Layer(n_inputs=1, n_outputs=1, ansatz=Ansatz(), backend=backend, shots=3000)
layers = [layer1]
network = QNN(layers)

In [93]:
network = QNN(layers)
for i in range(5):
    weight_gradients = network.backprop(np.array([0]), 0.3)
    y1 = network.a[-1]
    #weight_gradients2 = network.backprop(np.array([0.5]), 0.7)
    #y2 = network.a[-1]
    
    #weight_gradients = [(w1+w2)/2 for w1, w2 in zip(weight_gradients1, weight_gradients2)]

    print(y1)
    network.update(weight_gradients, 0.5)

[0.30166667]
[0.29933333]
[0.28566667]
[0.31133333]
[0.306]


In [94]:
print(layer1.weights)

[[0.99327247]
 [2.98735463]]
